In [9]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [10]:
#ee.Authenticate()
#geemap.update_package()

In [11]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])
##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)
##These are average lat/longs of the site (see GIS file); as such, need zonal spectral data (not point data)
##Lat/longs calculated in GA Biomass dataset Jupyter script

In [12]:
##landsat 5 surface reflectance images - GA LTER

gaCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2019,'year')) \
    .filter(ee.Filter.calendarRange(9, 10,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(gaCollectionLS5.size().getInfo())
gaimage0 = gaCollectionLS5.first()
props = geemap.image_props(gaimage0)
#props.getInfo()

68


In [13]:
##landsat 8 surface reflectance images - GA LTER
gaCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2015, 2016,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)


l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

print(gaCollectionLS8.size().getInfo())

27


In [14]:
#landsat 7 surface reflectance images - GA LTER - might not be able to use due to striping
gaCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2012,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

In [15]:
##get list of image ids in filtered collection for L5, L7, and L8
#gaCollectionLS5.aggregate_array('system:id').getInfo()
gaCollectionLS8.aggregate_array('system:id').getInfo()
#gaCollectionLS7.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C01/T1_SR/LC08_017038_20161121',
 'LANDSAT/LC08/C01/T1_SR/LC08_017038_20161105',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20151112',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20151112',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20161029',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20160911',
 'LANDSAT/LC08/C01/T1_SR/LC08_017038_20151018',
 'LANDSAT/LC08/C01/T1_SR/LC08_017038_20161020',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20160911',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20161013',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20151128',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20161029',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20150909',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20150909',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20160927',
 'LANDSAT/LC08/C01/T1_SR/LC08_017038_20160918',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20151011',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_20150925',
 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20160927',
 'LANDSAT/LC08/C01/T1_SR/LC08_017038_20161004',
 'LANDSAT/LC08/C01/T1_SR/LC08_016039_201

In [18]:
##Check sensing time of image - should be in GMT/UTC?
##Tide requirements: needs to be below 1.054 m at FORT PULASKI station (near Savannah) ID #8670870
##MHW = 2.173 m; MLW = 0.065 m

##landsat 5
ga00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20001024') ##TIDE = 0.2m ##some noise here; no spots near points 
ga01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20011027') ##TIDE = 0.5m
ga02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20021007') ##TIDE = 2.1m XXXXXX
ga03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20030915') ##TIDE = 2.2m XXXXXX ##clouds in oct, expanded to sep
ga04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20040901') ##TIDE = 2.0m XXXXXX ##clouds in oct, expanded to sep
ga05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20051015') ##TIDE = 0.74m
ga06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20061002') ##TIDE = 0.74m
ga07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20070910') ##TIDE = 0.85m ##clouds in oct, expanded to sep
ga08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20081030') ##TIDE = 2.1m XXXXXX
ga09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20091001') ##TIDE = 0.58m
ga10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20101029') ##TIDE = 1.93m XXXXXX
ga11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20111016') ##TIDE = 2.26m XXXXXX

##landsat 7
ga12 = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_017038_20121001') ##no more L7 XXXXXX look for L5/L8 overlap

##landsat 8
ga13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20131012') ##TIDE = 1.7m XXXXXX
ga14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20141008') ##TIDE = 1.25m XXXXXX

#ga15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20151018') ##TIDE = 2.4m XXXXXX
ga15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20151112') ##??????? NEED TO REDO

ga16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20161029') ##TIDE = 1.09m XXXXXX
#ga16 = ee.Image() ##TIDE = !!!!!!! LEFT OFF HERE

ga17 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20171016') ##TIDE = 0.28m

#ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20181003') ##TIDE = 1.15m XXXXXX
ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20180908') ##TIDE = 0.43m ##so many clouds, however; might be \
                                                            ## to use higher tide image? And calculate flooded pixels

ga19 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20190927') ##TIDE = 0.69m ##wrong scene in oct, expanded to sep

##determining cloud cover
cloudiness = ga00.get('CLOUD_COVER')
#print('CLOUD_COVER: ', cloudiness.getInfo())

#Map.addLayer(ga15, l8_param, "l82015")
#Map.addLayer(ga12, vis_param, "l72012")

props = geemap.image_props(ga16)
props.getInfo()

##Need to eliminate high tide images; use Fort Pulaski station? (Near Savannah; nearest harmonic station with data)

{'CLOUD_COVER': 2.39,
 'CLOUD_COVER_LAND': 0.1,
 'EARTH_SUN_DISTANCE': 0.993125,
 'ESPA_VERSION': '2_23_0_1a',
 'GEOMETRIC_RMSE_MODEL': 6.863,
 'GEOMETRIC_RMSE_MODEL_X': 4.829,
 'GEOMETRIC_RMSE_MODEL_Y': 4.877,
 'IMAGE_DATE': '2016-10-29',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_016038_20161029_20170219_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1487524478000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2016-10-29T15:54:58.3624200Z',
 'SOLAR_AZIMUTH_ANGLE': 156.849106,
 'SOLAR_ZENITH_ANGLE': 48.527077,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 16,
 'WRS_ROW': 38,
 'system:asset_size': '416.088978 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_016038_20161029',
 'system:index': 'LC08_016038_20161029',
 'system:time_end': '2016-10-29 15:54:58',


In [25]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(ga01, vis_param, "Landsat5SR")
#Map.addLayer(ga18, l8_param, "Landsat8SR")

Map.centerObject(ga_pts, 10)
#Map.set_plot_options(add_marker_cluster = True) ##not sure what this does


##add feature collection points for each plot
Map.addLayer(ga_pts)

Map
##what is with all the spots on ga00? Something wrong with image, salt & pepper noise?

Map(center=[31.434120927658846, -81.34324597505625], controls=(WidgetControl(options=['position', 'transparent…

In [22]:
##Water masking

##Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')

##Select the land/water mask.
datamask = hansenImage.select('datamask')

##Create a binary mask.
watermask = datamask.eq(1)

#watermasked00 = ga00.updateMask(watermask)
#Map.addLayer(watermasked00, vis_param, 'masked')

##Works! Although it is not perfect. Make sure to turn off background layers to see

In [23]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

##Map the function over image collection
collection = gaCollectionLS5 \
    .map(maskL5sr)

composite = collection.median() ##what exactly is this doing? median pixel value?

##Display the results.
#Map.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "cloudfree composite")
#Map.addLayer(ga_pts)

In [24]:
##water-, cloud-, and cloud shadow-masked images - GA LTER :D
gamasked00 = ee.Image(maskL5sr(ga00)).updateMask(watermask)
gamasked01 = ee.Image(maskL5sr(ga01)).updateMask(watermask)
gamasked02 = ee.Image(maskL5sr(ga02)).updateMask(watermask)
gamasked03 = ee.Image(maskL5sr(ga03)).updateMask(watermask)
gamasked04 = ee.Image(maskL5sr(ga04)).updateMask(watermask)
gamasked05 = ee.Image(maskL5sr(ga05)).updateMask(watermask)
gamasked06 = ee.Image(maskL5sr(ga06)).updateMask(watermask)
gamasked07 = ee.Image(maskL5sr(ga07)).updateMask(watermask)
gamasked08 = ee.Image(maskL5sr(ga08)).updateMask(watermask)
gamasked09 = ee.Image(maskL5sr(ga09)).updateMask(watermask)
gamasked10 = ee.Image(maskL5sr(ga10)).updateMask(watermask)
gamasked11 = ee.Image(maskL5sr(ga11)).updateMask(watermask)
gamasked12 = ee.Image(maskL5sr(ga12)).updateMask(watermask)
gamasked13 = ee.Image(maskL5sr(ga13)).updateMask(watermask)
gamasked14 = ee.Image(maskL5sr(ga14)).updateMask(watermask)
gamasked15 = ee.Image(maskL5sr(ga15)).updateMask(watermask)
gamasked16 = ee.Image(maskL5sr(ga16)).updateMask(watermask)
gamasked17 = ee.Image(maskL5sr(ga17)).updateMask(watermask)
gamasked18 = ee.Image(maskL5sr(ga18)).updateMask(watermask)
gamasked19 = ee.Image(maskL5sr(ga19)).updateMask(watermask)

##Check masked images:
Map.addLayer(gamasked00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "masked")
Map.addLayer(ga_pts)

##Creating an image collection:
ga_finalcollection = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked03, gamasked04, \
                                                    gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
                                                    gamasked10, gamasked11, gamasked12, gamasked13, gamasked14, \
                                                    gamasked15, gamasked16, gamasked17, gamasked18, gamasked19])

In [ ]:
##Exporting pixel values - any way to do zonal averages without manually selecting - GA LTER
in_fc = ga_pts

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'gamasked07.csv')
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting
#geemap.extract_values_to_points(in_fc, masked07, out_csv) ##projection error when exporting composites

In [ ]:
##Exporting pixel values - zonal statistics: Average 3x3 grid for Landsat \
##(radius = 30 or 90 m - is the 'scale' option in zonal_statistics radius???)

out_dir = os.path.expanduser('~/Downloads')
out_ga_stats = os.path.join(out_dir, 'ga_stats.csv')  

#geemap.zonal_statistics(in_value_raster = ga_finalcollection, in_zone_vector = ga_pts, \
                # out_file_path = out_ga_stats, statistics_type='MEAN', \
                # scale = 90)
        
        ##This ^^ is the right idea, but need to clean up bands selected, and still need to figure out a water mask
        ##Also: why the reflectance values so different than VA dataset: Has to do with masking params